In [1]:
import math

import torch

from script.NeuralNets.Networks import SequentialNN, ICNN, ICNNApproxMax, ICNNLogical
from script.settings import device, data_type
import script.DHOV.MultiDHOV as multidhov
from script.Verification.Verifier import SingleNeuronVerifier, MILPVerifier, DHOVVerifier
import gurobipy as grp
from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms import Compose, ToTensor, Normalize
from script.NeuralNets.ICNNFactory import ICNNFactory

In [2]:
def add_max_constr(model, neuron_name):
    neuron_var = model.getVarByName(neuron_name)
    model.setObjective(neuron_var, grp.GRB.MAXIMIZE)

def add_min_constr(model, neuron):
    neuron_var = model.getVarByName(neuron)
    model.setObjective(neuron_var, grp.GRB.MINIMIZE)

In [3]:
def optimize_model(model, neuron_name):
    model.update()
    model.optimize()
    if model.Status == grp.GRB.OPTIMAL:
        print("opt value: {}".format(model.getVarByName(neuron_name).getAttr("x")))

In [4]:
def icnn_model(icnn, nn, input_x, eps, layer_index, from_neuron, to_neuron, print_log=False):
    m = grp.Model()
    if not print_log:
        m.Params.LogToConsole = 0

    input_flattened = torch.flatten(input_x)
    bounds_affine_out, bounds_layer_out = nn.calculate_box_bounds(
        [input_flattened.add(-eps), input_flattened.add(eps)])

    parameter_list = list(nn.parameters())

    input_size = len(parameter_list[2*(layer_index-1)])
    lb = bounds_layer_out[layer_index-1][0].detach().cpu().numpy()
    ub = bounds_layer_out[layer_index-1][1].detach().cpu().numpy()
    in_var = m.addMVar(input_size, lb=-float("inf"), ub=float("inf"), name="icnn_var")

    low = bounds_layer_out[layer_index - 1][0][from_neuron: to_neuron]
    up = bounds_layer_out[layer_index - 1][1][from_neuron: to_neuron]
    low = torch.zeros_like(low, dtype=data_type).to(device) - 1000
    up = torch.zeros_like(low, dtype=data_type).to(device) + 1000
    constraint_bounds_affine_out, constraint_bounds_layer_out = icnn.calculate_box_bounds([low, up])
    icnn.add_max_output_constraints(m, in_var[from_neuron: to_neuron], constraint_bounds_affine_out, constraint_bounds_layer_out)

    return m

In [5]:
"""W1 = [1. 1.; 1. -1.]
    b1 = [0., 0.]
    W2 = [1. 1.; 1. -1.]
    b2 = [-0.5, 0.]
    W3 = [-1. 1.; 1. 1.]
    b3 = [3., 0.] """

"""nn = SequentialNN([2, 2, 2, 2])

with torch.no_grad():
    parameter_list = list(nn.parameters())
    parameter_list[0].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[1].data = torch.tensor([0, 0], dtype=data_type).to(device)
    parameter_list[2].data = torch.tensor([[1, 1], [1, -1]], dtype=data_type).to(device)
    parameter_list[3].data = torch.tensor([-0.5, 0], dtype=data_type).to(device)
    parameter_list[4].data = torch.tensor([[-1, 1], [1, 1]], dtype=data_type).to(device)
    parameter_list[5].data = torch.tensor([3, 0], dtype=data_type).to(device)

test_image = torch.tensor([[0, 0]], dtype=data_type).to(device)"""

"""transform = Compose([ToTensor(),
                         Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
                        )

training_data = CIFAR10(root="../../cifar", train=True, download=True, transform=transform)
images, labels = training_data.__getitem__(0)
test_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(
    torch.tensor(labels), 0).to(dtype=data_type).to(device)

nn = SequentialNN([32 * 32 * 3, 1024, 512, 10])
nn.load_state_dict(torch.load("../../cifar_fc.pth", map_location=torch.device(device)), strict=False)
"""

transform = Compose([ToTensor(),
                         Normalize(0.5, 0.5)]
                        )

training_data = MNIST(root="../../mnist",
                      train=True,
                      download=True,
                      transform=transform)
images, labels = training_data.__getitem__(0)
test_image, test_label = torch.unsqueeze(images, 0).to(dtype=data_type).to(device), torch.unsqueeze(
    torch.tensor(labels), 0).to(dtype=data_type).to(device)

nn = SequentialNN([28*28*1, 100, 30, 10])
nn.load_state_dict(torch.load("../../mnist_fc.pth", map_location=torch.device('cpu')), strict=False)

parameter_list = list(nn.parameters())

"""nn = SequentialNN([50, 50, 50, 7])
test_image = torch.zeros((1, 50), dtype=data_type).to(device)
parameter_list = list(nn.parameters())"""

'nn = SequentialNN([50, 50, 50, 7])\ntest_image = torch.zeros((1, 50), dtype=data_type).to(device)\nparameter_list = list(nn.parameters())'

In [6]:
eps = 0.01
layer_index = 1
neuron_index = 0
neuron_name = "relu_var{}[{}]".format(2*layer_index, neuron_index)

Test for DHOV

In [67]:
group_size = 2
icnn_factory = ICNNFactory("logical", [5, 5, 1], force_positive_init=False, with_two_layers=False,
                               init_scaling=10, init_all_with_zeros=False)
#icnn_factory = ICNNFactory("standard", [5, 5, 1])

icnns, last_group_indices, fixed_neuron_per_layer_lower, fixed_neuron_per_layer_upper, bounds_affine_out, bounds_layer_out = \
    multidhov.start_verification(nn, test_image, icnn_factory, group_size, eps=eps, icnn_epochs=500,
                                     icnn_batch_size=3, sample_count=1000, sample_new=True, use_over_approximation=True, break_after=51,
                                     sample_over_input_space=False, sample_over_output_space=True, use_icnn_bounds=True,
                                     use_fixed_neurons=True, sampling_method="per_group_sampling",
                                     force_inclusion_steps=0, preemptive_stop=False, even_gradient_training=False,
                                     keep_ambient_space=True, data_grad_descent_steps=0, opt_steps_gd=100,
                                     train_outer=False, print_training_loss=True,
                                     should_plot="none", optimizer="adam", init_network=True, adapt_lambda="none")



approximation of layer: 0
    number of fixed neurons for current layer: 92
    layer progress, group 1 of 4 
=== Epoch: 0===
batch = 0, mean loss = 1.2816370075495223
batch = 1, mean loss = 1.2816370075495223
time per epoch: 0.2530515193939209
=== Epoch: 1===
batch = 0, mean loss = 1.3314186258637766
batch = 1, mean loss = 1.3314186258637766
time per epoch: 0.032999277114868164
=== Epoch: 2===
batch = 0, mean loss = 1.3191488859164855
batch = 1, mean loss = 1.3191488859164855
time per epoch: 0.031000614166259766
=== Epoch: 3===
batch = 0, mean loss = 1.283836804538065
batch = 1, mean loss = 1.283836804538065
time per epoch: 0.03100109100341797
=== Epoch: 4===
batch = 0, mean loss = 1.2532449631059832
batch = 1, mean loss = 1.2532449631059832
time per epoch: 0.03299832344055176
=== Epoch: 5===
batch = 0, mean loss = 1.2546358998334801
batch = 1, mean loss = 1.2546358998334801
time per epoch: 0.032000064849853516
=== Epoch: 6===
batch = 0, mean loss = 1.237018360289654
batch = 1, mean 

KeyboardInterrupt: 

In [53]:
print(last_group_indices)

[[0]]


In [54]:
print(bounds_affine_out[1][0])

tensor([-0.0757,  5.6189,  1.1482, -3.6142,  1.6317, -1.2987,  0.5409, -0.5313,
         1.9913,  0.7412,  1.2950,  3.1743,  2.3159, -1.3097, -0.9965,  5.6670,
         0.2652,  2.9580, -0.9048,  5.3704,  0.0706,  6.1707,  1.9918,  5.1470,
         7.3835, -0.7402,  4.4840,  4.4441,  1.3274,  3.7316],
       dtype=torch.float64, grad_fn=<CopySlices>)


In [55]:
print(bounds_affine_out[1][1])

tensor([ 0.6429,  6.1889,  1.7723, -3.1887,  2.3423, -0.7422,  1.2457, -0.0882,
         2.6099,  1.2459,  1.9281,  3.7094,  2.8397, -1.0436, -0.7669,  6.3403,
         1.1305,  3.5073, -0.6783,  5.9606,  0.5130,  6.7461,  2.5670,  5.8033,
         8.0480, -0.4741,  4.9826,  5.1774,  1.6728,  4.3185],
       dtype=torch.float64, grad_fn=<CopySlices>)


In [56]:
input_flattened = torch.flatten(test_image)
simple_bounds_affine_out, simple_bounds_layer_out = nn.calculate_box_bounds([input_flattened.add(-eps), input_flattened.add(eps)])
print(simple_bounds_affine_out[1][0])
print(simple_bounds_affine_out[1][1])

tensor([-1.0273,  4.8911,  0.4758, -4.1546,  0.6518, -2.1435, -0.4091, -1.1941,
         1.1518,  0.0534,  0.4475,  2.2823,  1.6934, -1.8334, -1.4669,  4.6682,
        -0.6848,  2.2005, -1.2830,  4.6710, -0.6092,  5.4843,  1.2427,  4.1688,
         6.4462, -1.1600,  3.7549,  3.5614,  0.7072,  2.9282],
       dtype=torch.float64, grad_fn=<AddBackward0>)
tensor([ 1.5903,  6.9082,  2.4571, -2.6379,  3.3335,  0.0854,  2.2013,  0.5879,
         3.4414,  1.9512,  2.7689,  4.5998,  3.4715, -0.5237, -0.3061,  7.3616,
         2.0802,  4.2680, -0.2947,  6.6573,  1.2007,  7.4136,  3.3206,  6.7802,
         8.9711, -0.0464,  5.7120,  6.0631,  2.2935,  5.0945],
       dtype=torch.float64, grad_fn=<AddBackward0>)


In [57]:
dhov_model = icnn_model(icnns[layer_index][0], nn, test_image, eps, layer_index, 0, 1, print_log=True)
dhov_model.update()

In [58]:
icnn_neuron_name = "icnn_var[{}]".format(neuron_index)

In [59]:
dhov_copy = dhov_model.copy()
dhov_copy.Params.LogToConsole = 1
add_min_constr(dhov_copy, icnn_neuron_name)
dhov_copy.update()
all_var = dhov_copy.getVars()
optimize_model(dhov_copy, icnn_neuron_name)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 39 rows, 131 columns and 86 nonzeros
Model fingerprint: 0x73f1282a
Model has 3 general constraints
Variable types: 131 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-04, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e-01, 3e+03]
  RHS range        [1e-02, 3e+00]
Presolve removed 32 rows and 126 columns
Presolve time: 0.00s
Presolved: 7 rows, 5 columns, 16 nonzeros
Variable types: 4 continuous, 1 integer (1 binary)
Found heuristic solution: objective -0.0078686

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: -0.00786857 

Optimal solution found (tolerance 1.00e-04)
Best objective -7.868574787573e-03, best bo

In [60]:
dhov_copy = dhov_model.copy()
dhov_copy.Params.LogToConsole = 0
add_max_constr(dhov_copy, icnn_neuron_name)
optimize_model(dhov_copy, icnn_neuron_name)

opt value: 0.6428877384264218


Test for SNV

In [17]:
snv_verifier = SingleNeuronVerifier(nn, test_image, eps, print_log=False)
snv_verifier.generate_constraints_for_net()
snv_model = snv_verifier.model
snv_model.update()

In [18]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_min_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 0.0


In [19]:
snv_copy = snv_model.copy()
snv_copy.Params.LogToConsole = 0
add_max_constr(snv_copy, neuron_name)
optimize_model(snv_copy, neuron_name)

opt value: 0.9722473212684741


Test for MILP

In [20]:
neuron_name = "relu_var{}[{}]".format(2*layer_index, 0)
milp_verifier = MILPVerifier(nn, test_image, eps, print_log=False)
milp_verifier.generate_constraints_for_net(until_layer_neuron=[layer_index, neuron_index])
milp_model = milp_verifier.model
milp_model.update()

"""all_var = milp_model.getVars()
for var in all_var:
    print(var)"""


'all_var = milp_model.getVars()\nfor var in all_var:\n    print(var)'

In [21]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_min_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 0.0


In [22]:
milp_copy = milp_model.copy()
milp_copy.Params.LogToConsole = 0
add_max_constr(milp_copy, neuron_name)
optimize_model(milp_copy, neuron_name)

opt value: 0.5622137652362753
